In [11]:
#https://medium.com/@pallawi.ds/semantic-segmentation-with-u-net-train-and-test-on-your-custom-data-in-keras-39e4f972ec89
#Keras loss functions
#https://github.com/maxvfischer/keras-image-segmentation-loss-functions
#https://github.com/maxvfischer/keras-image-segmentation-loss-functions/blob/master/losses/multiclass_losses.py#L107
import os
import sys
from pathlib import Path
utilsPath = os.path.join(Path(os.getcwd()).parent.absolute(),'utils')
if utilsPath not in sys.path:
    sys.path.append(utilsPath)
#print(sys.path)
from getTiles import image_gen
from getTiles import read_image
from lossFunctions import iou_coef, DiceLoss, weighted_categorical_crossentropy
import random
import glob
import warnings
import numpy as np
import pandas as pd
import matplotlib
import datetime

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.regularizers import l2
from keras.layers.merge import concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras import backend as K
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers import SGD

import keras
import tensorflow as tf
import cv2
import rasterio
#import tensorflow.keras.backend as K

from typing import Callable, Union

from sklearn.metrics import confusion_matrix, accuracy_score, multilabel_confusion_matrix, classification_report

import matplotlib.pyplot as plt
from matplotlib import colors
from rasterio.plot import show


In [12]:
#Define training data and test data

seed = 42
random.seed = seed
np.random.seed = seed

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

num_classes = 8

labelFilePath = '../inputs/labeledTilesValidYears/plot_{}_{}.tif'
bandFilePath = '../inputs/landsatTiles30pPostYear_3/Landsat7_{}_{}.tif'

fileNames = glob.glob('../inputs/labeledTilesValidYears/*.tif')
fileNames.sort()

plotIDYears = [[int(x.split('_')[1]),int(x.split('_')[2].split('.')[0])] for x in fileNames]

plotIDS = np.unique([plotIDYears[i][0] for i in np.arange(len(plotIDYears))])
np.random.shuffle(plotIDS)


#Split into training and validation
trainingPlotIDs = plotIDS[:int(len(plotIDS)*0.7)]
validationPlotIDs = plotIDS[int(len(plotIDS)*0.7):]

trainingPlots = [x for x in plotIDYears if x[0] in trainingPlotIDs]
validationPlots = [x for x in plotIDYears if x[0] in validationPlotIDs]
print(len(trainingPlots), len(validationPlots))


474 219


In [16]:
#set parameters
n_classes=8
n_features = 4
image_size = 64
batch_size=64
bands = 'all'


In [17]:
#Model name to save
modelName = 'unet3'

#Define if we want to reinialize a new model
newModel = False


# Define weights for classes!
classnames = ['No Loss','Hard commodities', 'Forest products', 'Other disturbances',
                'Soft commodities', 'Settlements/infrastructure', 'Fires','Hansen Mistake']
class_weights = np.array([2, 80, 60, 100,
                          100, 90, 100,100]).astype(float)


model = keras.models.load_model('models/{}'.format(modelName), custom_objects = {"loss":weighted_categorical_crossentropy(class_weights),"iou_coef": iou_coef, "weighted_categorical_crossentropy": weighted_categorical_crossentropy(class_weights)})



In [20]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

classnames = ['No\nloss','Hard\ncommodities', 'Forest\nproducts', 'Other\ndisturbances',
                'Soft\ncommodities', 'Settlements/\ninfrastructure', 'Fires','Hansen\nmistake']
classcolors = ['#ffffff','#FCABAB','#93D896','#C5E4FC','#FBFD38','#BABABA','#FC3B26','#3540c8']


modelPath = 'models/{}'.format(modelName)
# #Load model
model = keras.models.load_model(modelPath, custom_objects = {"loss":weighted_categorical_crossentropy(class_weights),"iou_coef": iou_coef, "weighted_categorical_crossentropy": weighted_categorical_crossentropy(class_weights)})


predictionFilePath = '../outputs/{}/plot_{}_{}.tif'


random.seed = seed
np.random.seed = seed
#ids = np.arange(len(validationPlots))#np.random.randint(0, high=len(validationPlots), size=len(validationPlots))#[1,2,3,5,7,9,10,200,300,200,40,20,150]
ids= [[363, 2010],[79, 2017],[63, 2001],[23, 2006],[439, 2010]]

for idy in ids:
    #validationID = validationPlots[idy] 
    validationID = idy
    validationXRaw, label = read_image(validationID, labelFilePath, bandFilePath,bands=bands)
    validationX = np.zeros((1, image_size, image_size, n_features))
    validationX[0,:,:,:] = validationXRaw
    
    ynew = model.predict(validationX)
    ynew = ynew[0]
    label = label[0]
    ynew = np.argmax(ynew, axis=-1)

# #     ##Code block to filter to only loss
#     ynew[:,:,0] = 0
#     ynew = np.argmax(ynew,axis=-1)
#     ynew = np.where(label==0, 0, ynew) 
    
    
    
    labelsrc = rasterio.open(labelFilePath.format(validationID[0],validationID[1]))
    profile = labelsrc.profile


    with rasterio.open(predictionFilePath.format(modelName,validationID[0],validationID[1]),
                                                 'w', **profile) as dst:
        dst.write(ynew.astype(rasterio.uint8), 1)
    
